In [1]:
import gc

In [1]:
import wandb
print(wandb.__version__)

0.18.5


In [2]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.5.1
0.20.1


In [3]:
import torch
print(torch.cuda.is_available())  # True여야 GPU 사용 가능

True


In [4]:
import wandb
from ultralytics import YOLO
import torch
from torch.quantization import quantize_dynamic

In [5]:
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "yolo_quan_local.ipynb"

In [ ]:
# WandB에 로그인
wandb.login(key='debec7acbfa814ac1d35013a84ac13808d992cd0')

In [ ]:
# Sweep 구성설정
sweep_config = {
    'name': 'yolo_sweep',
    'method': 'bayes',  # 또는 "grid"를 사용할 수 있습니다.
    'metric': {'name': 'box_loss', 'goal': 'minimize'},
    'parameters': {
        'batch_size': {'values': [16, 32]}, # 64 x
        'learning_rate': {'min': 1e-5, 'max': 1e-2},
        'epochs': {'values': [10, 20, 30, 40, 50]},
        "patience": {"value": 3}  # Early stopping을 위한 patience 추가
    }
}

# Sweep ID 생성
sweep_id = wandb.sweep(sweep_config, project="yolo11_re")

def train():
    # WandB 초기화
    wandb.init(project="yolo11_re", entity="icejoy000524-student")

    # Sweep 매개변수 가져오기
    config = wandb.config

    # YOLOv11 모델 로드
    model = YOLO("yolo11n.pt")  # YOLOv11 모델 가중치 파일 경로

    best_mAP = 0
    epochs_without_improvement = 0

    # 훈련 설정
    for epoch in range(config.epochs):
        train_results = model.train(
            data="C:/Users/희정/NIPAPROJECT/yolodata/yolo_split/yolo.yaml",  # 데이터셋 YAML 파일 경로
            epochs=1,  # 에포크 수를 1로 설정하여 for 루프에서 개별적으로 훈련
            imgsz=416,  # 이미지 크기 640->416
            batch=config.batch_size,  # Sweep에서 정의한 배치 사이즈
            lr0=config.learning_rate,
            device="cuda" if torch.cuda.is_available() else "cpu",  # 사용 가능한 장치
        )

        # 결과에서 메트릭 값 가져오기
        metrics = train_results.results_dict
        val_precision = metrics['metrics/precision(B)']  # 정밀도
        val_recall = metrics['metrics/recall(B)']        # 재현율
        val_map50 = metrics['metrics/mAP50(B)']          # mAP@50
        val_map50_95 = metrics['metrics/mAP50-95(B)']    # mAP@50:95
        val_fitness = metrics['fitness']                   # fitness score
                 
        # mAP@50 값을 current_mAP로 사용
        current_mAP = val_map50 

        # WandB에 메트릭 기록
        wandb.log({
            "epoch": epoch,
            "val_precision": val_precision,
            "val_recall": val_recall,
            "val_map50": val_map50,
            "val_map50_95": val_map50_95,
            "val_fitness": val_fitness,
            "val_map": current_mAP
        })


        # Early stopping 로직
        if current_mAP > best_mAP:
            best_mAP = current_mAP
            epochs_without_improvement = 0  # 성능 개선 시 카운트 초기화

            print(f"New best mAP: {best_mAP} at epoch {epoch}")
            
            model_filename = f"best_yolo_model_{wandb.run.id}.pt"
            wandb.save(model_filename)   # WandB에 모델 업로드
            
        else:
            epochs_without_improvement += 1
            print(f"No improvement in mAP for {epochs_without_improvement} epochs.")

            if epochs_without_improvement >= config.patience:
                print(f"Early stopping triggered after {epochs_without_improvement} epochs without improvement.")
                break  # 훈련 종료

        # GPU 메모리 해제
        torch.cuda.empty_cache()
        gc.collect()  # 가비지 컬렉션으로 CPU 메모리 비우기

    # WandB 훈련 완료 로그 기록
    wandb.finish()

# Sweep 에이전트 실행
wandb.agent(sweep_id, function=train, count=10)